In [1]:
from dispatches.util import generate_syn_realizations

# These are DA only Prices
lmp_data = generate_syn_realizations(pkl_filename="ARMA_Model/output/arma.pk", set_years=[2018], n_scenarios=10)

PluginFactory: No installed plugins detected.
InputData: Using param spec "pickledROM" to read XML node "ROM.
( 1158.10 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
( 1158.11 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "2018" (1 / 1)
( 1159.26 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
( 1159.26 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "2018" (1 / 1)
( 1159.38 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
( 1159.38 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "2018" (1 / 1)
( 1159.49 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
( 1159.49 sec) Interp. Cluster ROM      : DEBUG           ->  ... evaluating macro step "2018" (1 / 1)
( 1159.68 sec) Interp. Cluster ROM      : DEBUG           -> Evaluating interpolated ROM ...
( 1159.70 sec

# Sizing from Conceptual Design Optimization

For the following scenario:
 - wind size is fixed
 - only the DA prices are used
 - hydrogen price is $3/kg
 - price cap is $7550/MWh

The optimal design was:
 - 357.10000 MW, 375.894737 MWh Battery
 - 190.1 MW PEM
 - 0.0 MW H2 Tank and Turbine
 - -1.392221 $Bil NPV

In [5]:
import numpy as np
import multiprocessing as mp
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from dispatches.case_studies.renewables_h2_case.wind_battery_hydrogen_flowsheet import wind_battery_hydrogen_optimize
from dispatches.case_studies.renewables_case.RE_flowsheet import default_input_params

price_cap = 7550
h2_price = 3
batt_mw = 357.1
batt_mwh = 375.89
pem_mw = 190.1
tank_tonh2 = 0
turb_mw = 0

syn_prices = []
for prices in lmp_data.values():
    syn_prices.append(np.clip(prices[2018], 0, price_cap))

def run_design(prices):
    input_params = default_input_params.copy()
    input_params['design_opt'] = False
    input_params['build_add_wind'] = 0
    input_params['opt_mode'] = "pricetaker"
    input_params['h2_price_per_kg'] = h2_price
    input_params["batt_mw"] = batt_mw
    input_params["batt_mwh"] = batt_mwh
    input_params["turb_conv"] = 20
    input_params["tank_size"] = tank_tonh2
    input_params["turb_mw"] = turb_mw
    input_params['LMPs'] = prices
    des_res = wind_battery_hydrogen_optimize(
        # n_time_points=24 * 7, 
        n_time_points=len(prices), 
        input_params=input_params, verbose=False, plot=False)
    res = {**input_params, **des_res[0]}
    res.pop("LMPs")
    res.pop("design_opt")
    res.pop("extant_wind")
    res.pop("wind_resource")
    res.pop("pyo_model")
    return res, des_res[1]

In [9]:
res = run_design(syn_prices[0])

[+   0.00] Beginning the formulation of the multiperiod optimization problem.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 component keys that are not exported as part of the NL file.
    Skipping.
    contains 6 compone

In [8]:
#%%
import multiprocessing as mp

#%%
if __name__ ==  '__main__': 
    p = mp.Pool(processes=6)
    res = p.starmap(run_design, syn_prices)
    p.close()
    p.join()

Process SpawnPoolWorker-25:
Process SpawnPoolWorker-24:
Traceback (most recent call last):
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'run_design' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/dguittet/miniconda3/envs/dispatches/lib/python3.8/multiprocessing/process.py"

KeyboardInterrupt: 

In [ ]:
timeseries

,Total Wind Generation [MW],Total Power Output [MW],Wind Power Output [MW],Wind Power to Battery [MW],State of Charge [MWh],Battery Power Output [MW],Wind Power to PEM [MW],PEM H2 Output [kg],Tank H2 Input [kg],H2 Sales [kg],"Turbine H2 Input [kg]""",Tank Holdup [kg],Turbine Power Output [MW]
0,54.333333,0.000000,0.000000,0.000000,339.245,0.0000,54.333333,988.721227,988.721227,988.721227,0.0,0.0,0.0
1,14.700000,0.000000,0.000000,0.000000,339.245,0.0000,14.700000,267.500651,267.500651,267.500651,0.0,0.0,0.0
2,7.716667,0.000000,0.000000,0.000000,339.245,0.0000,7.716667,140.422677,140.422677,140.422677,0.0,0.0,0.0
3,13.700000,0.000000,0.000000,0.000000,339.245,0.0000,13.700000,249.303328,249.303328,249.303328,0.0,0.0,0.0
4,18.491667,0.000000,0.000000,0.000000,339.245,0.0000,18.491667,336.498835,336.498835,336.498835,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,747.100000,747.100000,747.100000,0.000000,339.245,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
164,770.583333,332.009649,332.009649,38.573684,375.890,0.0000,400.000000,7278.929280,7278.929280,7278.929280,0.0,0.0,0.0
165,761.791667,718.887167,361.791667,0.000000,0.000,357.0955,400.000000,7278.929280,7278.929280,7278.929280,0.0,0.0,0.0
166,678.566667,678.566667,678.566667,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
